# Data smells
Any time you are given a dataset from anyone, you should immediately be suspicious. Is this data what I think it is? Does it include what I expect? Is there anything I need to know about it? Will it produce the information I expect?

One of the first things you should do is give it the smell test. 

Failure to give data the smell test [can lead you to miss stories and get your butt kicked on a competitive story](https://source.opennews.org/en-US/learning/handling-data-about-race-and-ethnicity/).

Let's look at some campus crime data. You can [get it here](https://www.dropbox.com/s/avt894qyqhs1gp1/unlcrime.csv?dl=0) or find it in the Data folder in the class GitHub repository.

In [2]:
import agate

import warnings
warnings.filterwarnings('ignore')

unlcrimes = agate.Table.from_csv('../../Data/unlcrime.csv')

With data smells, we're trying to find common mistakes in data. [For more on data smells, read the GitHub wiki post that started it all](https://github.com/nikeiubel/data-smells/wiki/Ensuring-Accuracy-in-Data-Journalism). The common mistakes we're looking for are:

* Missing data
* Gaps in data
* Wrong type of data
* Outliers 
* Sharp curves
* Conflicting information within a dataset
* Conflicting information across datasets
* Wrongly derived data
* External inconsistency
* Wrong spatial data
* Unusuable data, including non-standard abbreviations, ambigious data, extraneous data, inconsistent data

Not all of these data smells are detectable in code. You may have to ask people about the data. You may have to compare it to another dataset yourself. Does the agency that uses the data produce reports from the data? Does your analysis match those reports? That will expose wrongly derived data, or wrong units, or mistakes you made with inclusion or exclusion. 

But with several of these data smells, we can do them first, before we do anything else. First, let's look at **Wrong Type Of Data**. We can sniff that out by simply printing the table structure that Agate has discovered for us.

In [3]:
print(unlcrimes)

| column         | data_type |
| -------------- | --------- |
| Case #         | Text      |
| Incident Code  | Text      |
| Reported       | DateTime  |
| Case Status    | Text      |
| Start Occurred | DateTime  |
| End Occurred   | DateTime  |
| Building       | Text      |
| Location       | Text      |
| Stolen         | Number    |
| Damaged        | Number    |
| Description    | Text      |



For this, things seem to be good. Case #, because it has number in the name, might tempt you to think it's a number, but it's not anything you're going to do math on, so it's text. Dates appear to be dates, things that aren't numbers appear to be text, and the two dollar figures appear to be counted as numbers. That's a good start.

The second smell we can find in Agate is Missing Data. We can do that through a series of Group By and Count steps. Let's start with Incident Codes.

In [15]:
codes = unlcrimes.group_by('Incident Code')
code_counts = codes.aggregate([
    ('count', agate.Count())
])

# We need to sort it in descending order, z to a, becase Agate interprets blanks as after z
code_counts = code_counts.order_by('Incident Code', reverse=True)

In [14]:
code_counts.print_table()

| Incident Code        | count |
| -------------------- | ----- |
|                      |     1 |
| WEAPONS вЂ“ OTHER    |    10 |
| WEAPONS - ILLEGAL... |     1 |
| WEAPONS - FELON I... |     3 |
| WEAPONS - DISCHAR... |     3 |
| WEAPONS - CONCEALED  |     7 |
| WARRANT SERVICE      |     4 |
| VANDALISM - OTHER    |   392 |
| VANDALISM - BY GR... |   106 |
| UNL POLICY VIOLAT... |    30 |
| UNL POLICY VIOLAT... |    12 |
| TRESPASSING          |   311 |
| TRAFFIC - SUSPEND... |   707 |
| TRAFFIC - OTHER      |    54 |
| TRAFFIC - IMPROPE... |     2 |
| TRAFFIC - DIRECTION  |     1 |
| TELEPHONE - THREA... |    15 |
| TELEPHONE - OTHER    |    56 |
| TELEPHONE - OBSCE... |     6 |
| SUSPICIOUS VEHICLE   |     1 |
| ...                  |   ... |


And there's that last one. It's blank. There is missing data. But it's only 1 record of thousands, so it's not going to matter in the grand scheme. If that had been larger, we would have to do some more reporting.

Let's now look at **Gaps in Data**. It's been my experience that gaps in data often have to do with time, so let's first look at crimes by year, so we can see if there's a year with big jumps in reported crime. You'd expect the number to change, but not by huge amounts. Huge change would indicate, more often than not, that the data is missing. 

In [7]:
crimes_with_years = unlcrimes.compute([
    ('reported_year', agate.Formula(agate.Text(), lambda row: '%s' % row['Reported'].year))
])

In [9]:
years = crimes_with_years.group_by('reported_year')
year_counts = years.aggregate([
    ('count', agate.Count())
])
year_counts.print_table()

| reported_year | count |
| ------------- | ----- |
| 2010          |   185 |
| 2011          | 1,948 |
| 2012          | 1,881 |
| 2013          | 2,052 |
| 2014          | 1,920 |
| 2015          | 1,750 |


So, something doesn't look right here. 185 crimes in 2010? Not likely. And what about 2015? A ~200 incident drop wouldn't be *that* unheard of -- look at 2012 to 2013 -- but I'm suspicious. Did we get the whole of 2015? Let's check.

In [10]:
crimes_with_monthyears = unlcrimes.compute([
    ('reported_monthyear', agate.Formula(agate.Text(), lambda row: '%s/%s' % (row['Reported'].month, row['Reported'].year)))
])

In [12]:
monthyears = crimes_with_monthyears.group_by('reported_monthyear')
monthyear_counts = monthyears.aggregate([
    ('count', agate.Count())
])
monthyear_counts.print_bars('reported_monthyear', 'count', width=60)

reported_monthyear count
11/2010               56 ▓░░░░░░                            
12/2010              129 ▓░░░░░░░░░░░░░░░                   
1/2011               157 ▓░░░░░░░░░░░░░░░░░░                
2/2011               168 ▓░░░░░░░░░░░░░░░░░░░               
3/2011               157 ▓░░░░░░░░░░░░░░░░░░                
4/2011               184 ▓░░░░░░░░░░░░░░░░░░░░░             
5/2011               130 ▓░░░░░░░░░░░░░░░                   
6/2011               124 ▓░░░░░░░░░░░░░░                    
7/2011               137 ▓░░░░░░░░░░░░░░░░                  
8/2011               183 ▓░░░░░░░░░░░░░░░░░░░░░             
9/2011               242 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░       
10/2011              175 ▓░░░░░░░░░░░░░░░░░░░░              
11/2011              179 ▓░░░░░░░░░░░░░░░░░░░░              
12/2011              112 ▓░░░░░░░░░░░░░                     
1/2012               194 ▓░░░░░░░░░░░░░░░░░░░░░░            
2/2012               147 ▓░░░░░░░░░░░░░░░░░                 

So, it appears we were right to suspect of 2010 and 2015. Did 2015 end magically in November? Nope. We have incomplete data. 

## Assignment

What about Location and Building? Or Stolen and Damaged? Is there missing data? Is there wrong data? How normalized is that data? Are there outliers? What steps in Agate should you take to find out? Continue this notebook below and use tools in Agate to find out if anything is clearly wrong with the fields above -- [a whole list of descriptive statistics is here](http://agate.readthedocs.io/en/1.6.0/cookbook/statistics.html).